In [42]:
import pandas as pd
import numpy as np
from sklearn import tree

In [3]:
#botdata = pd.read_csv('botAccounts.csv')
#gooddata= pd.read_csv('GoodAccounts.csv', encoding = 'ISO-8859-1')

In [4]:
testbot = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/bots_data.csv", encoding = 'ISO-8859-1')
testgood = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/nonbots_data.csv", encoding = 'ISO-8859-1')

In [6]:
df.head()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listedcount,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,3.982732e+09,3982731976,mcgucket_bot,NaN,A bot that tweets every line said by Fiddlefor...,NaN,1129,7,2,Sat Oct 17 22:35:31 +0000 2015,0,False,23557,en,"{'retweeted': False, 'is_quote_status': False,...",False,False,False,McGucket bot,1
1,8.410000e+17,8.41E+17,BowieK66,NaN,NaN,NaN,0,22,0,13/03/2017 22:21,0,False,1,en,Status(_api=<tweepy.api.API object at 0x101927...,True,True,False,MJ Kuhn,1
2,2.768375e+09,2768374981,ducknoteprice,NaN,NaN,NaN,3,0,3,Mon Aug 25 23:17:59 +0000 2014,0,False,1050,en,{'created_at': 'Fri Feb 26 17:01:15 +0000 2016...,True,False,False,duckNote,1
3,3.304189e+09,3304189373,robotrecipes,robot kitchen,tasty recipes for robot // not for human // a ...,http://t.co/PdagJGqVMR,505,13,49,Sat May 30 23:42:16 +0000 2015,0,False,5109,en,"{u'contributors': None, u'truncated': False, u...",False,False,False,robot recipes,1
4,3.224289e+09,3224289024,everyumlaut,NaN,bot by @dbaker_h,NaN,15,0,11,Sat May 23 14:39:21 +0000 2015,0,False,31365,en-gb,"{u'contributors': None, u'truncated': False, u...",True,False,False,ÌÇvÌÇrÌÀ Ì_mlÌ_Ì_¼Ñ,1


In [22]:
testgood.shape

(1176, 20)

In [23]:
df.shape

(2232, 20)

In [24]:
#modify the column name to fit
def adjust_column_name(df):
    for x in df.columns.values:
        df = df.rename(columns={x: x.lower()})
    df = df.rename(columns={'listed_count' : 'listedcount'})

In [25]:
# split the dataset to training & testing
from sklearn.cross_validation import train_test_split
train, test = train_test_split(df, test_size = 0.2)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

In [27]:
df = testbot.append(testgood).fillna("")

In [30]:
def nb_dt(nb_column_name):
    train, test = train_test_split(df, test_size = 0.2)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count','default_profile','default_profile_image','verified']
    #numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count','nb_result']
    count_vect = CountVectorizer()
    
    def convert(column):
        X_word_counts = count_vect.fit_transform(column)
        count_vect.vocabulary_.get(u'algorithm')
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    def convert2(column):
        X_word_counts = count_vect.transform(test[nb_column_name])
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    #Train
    clf = tree.DecisionTreeClassifier(random_state=0)
    M_clf = MultinomialNB().fit(convert(train[nb_column_name]), train['bot'])
    train['nb_result'] = M_clf.predict(convert(train[nb_column_name]))
    clf.fit(train[numfeatures],train["bot"])
    #Test
    test['nb_result'] = M_clf.predict(convert2(train[nb_column_name]))
    
    #Evaluate
    accuracy = accuracy_score(clf.predict(test[numfeatures]),test['bot'])
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [accuracy, precison, recall, f1, roc]

In [32]:
for _ in range(10):
    print(nb_dt('description'))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.83668903803131989, 0.80930232558139537, 0.84466019417475724, 0.82660332541567694, 0.83726785642347823]
[0.83892617449664431, 0.839622641509434, 0.82407407407407407, 0.83177570093457942, 0.83844396344396344]
[0.83892617449664431, 0.81304347826086953, 0.8657407407407407, 0.83856502242152453, 0.83979677729677726]
[0.85011185682326618, 0.81481481481481477, 0.86699507389162567, 0.84009546539379476, 0.85153032383105876]
[0.84787472035794187, 0.85427135678391963, 0.8133971291866029, 0.83333333333333337, 0.84577419484540217]
[0.85011185682326618, 0.83856502242152464, 0.85779816513761464, 0.84807256235827666, 0.85029646248147106]
[0.87024608501118572, 0.85116279069767442, 0.87559808612440193, 0.8632075471698113, 0.87057215230589846]
[0.83668903803131989, 0.83255813953488367, 0.82870370370370372, 0.83062645011600922, 0.83642977392977391]
[0.87695749440715887, 0.88317757009345799, 0.86301369863013699, 0.87297921478060048, 0.87668228791155967]
[0.8232662192393736, 0.79090909090909089, 0.8405797

In [36]:
def dt():
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count','default_profile','default_profile_image','verified']
    train, test = train_test_split(df, test_size = 0.2)
    clf = tree.DecisionTreeClassifier(random_state=0)
    clf.fit(train[numfeatures],train["bot"])
    accuracy = accuracy_score(clf.predict(test[numfeatures]),test['bot'])
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [accuracy, precison, recall, f1, roc]

In [37]:
for _ in range(10):
    print(dt())

[0.83221476510067116, 0.84507042253521125, 0.81081081081081086, 0.82758620689655182, 0.83207207207207201]
[0.84787472035794187, 0.83173076923076927, 0.83980582524271841, 0.83574879227053134, 0.84728880473754165]
[0.84787472035794187, 0.84079601990049746, 0.82439024390243898, 0.83251231527093594, 0.84607941947188059]
[0.80089485458612975, 0.84313725490196079, 0.75109170305676853, 0.79445727482678974, 0.80215135611554023]
[0.83445190156599558, 0.85167464114832536, 0.80542986425339369, 0.82790697674418612, 0.83413086133023662]
[0.86353467561521258, 0.86036036036036034, 0.86425339366515841, 0.86230248306997748, 0.86354262603611898]
[0.84116331096196872, 0.8232323232323232, 0.81909547738693467, 0.82115869017632237, 0.83898322256443514]
[0.8232662192393736, 0.80829015544041449, 0.78787878787878785, 0.79795396419437337, 0.81964220518437381]
[0.85011185682326618, 0.81990521327014221, 0.85643564356435642, 0.83777239709443108, 0.85066680137401485]
[0.83892617449664431, 0.85507246376811596, 0.808

In [24]:
for attr in literal_eval(x):
    df[attr] = (ast.literal_eval(df['status'])).astype(int)

In [22]:
import ast

In [113]:
df['name_has_bot'] = df['name'].str.contains('bot')

In [118]:
df.columns.values

array(['id', 'id_str', 'screen_name', 'location', 'description', 'url',
       'followers_count', 'friends_count', 'listedcount', 'created_at',
       'favourites_count', 'verified', 'statuses_count', 'lang', 'status',
       'default_profile', 'default_profile_image', 'has_extended_profile',
       'name', 'bot', 'name_has_bot'], dtype=object)